In [ ]:
from pybricksdev.ble import find_device
from pybricksdev.connections import BLEConnection
from asyncio import gather, sleep
import logging

class Remote(BLEConnection):
    
    CHAR_UUID = '00001624-1212-efde-1623-785feabcd123'
    
    class Keypad():
        def __init__(self):
            self.left_state = bytes((0, 0, 0))
            self.right_state = bytes((0, 0, 0))
            self.center_state = 0
        
        def pressed(self):
            # TODO: return Pybricks Button enum tuple
            return (self.center_state, self.left_state, self.right_state)

    def __init__(self):
        super().__init__(self.CHAR_UUID, self.CHAR_UUID, 20)

        self.buttons = self.Keypad()
    
    async def setup(self):
        # Thanks @bricknil for documenting the right values
        await remote.write(bytes([10, 0, 65, 0, 4, 1, 0, 0, 0, 1]))
        await remote.write(bytes([10, 0, 65, 1, 4, 1, 0, 0, 0, 1]))
        
        color = 6
        await remote.write(bytes([8, 0, 129, 52, 1, 81, 0, color]))
    
    def data_handler(self, sender, data):
        
        self.logger.debug(sender, data)
        
        # Parse change in button state
        if data[0:3] == bytes((7, 0, 69)):
            if data[3]:
                self.buttons.left_state = bytes(data[4:])
            else:
                self.buttons.right_state = bytes(data[4:])
        if data[0:4] == bytes((5, 0, 8, 2)):
            self.buttons.center_state = data[4]


address = await find_device('Handset')
print(address)
        
remote = Remote()
await remote.connect(address)
await remote.setup()


In [ ]:
# Print the button state a few times
for i in range(30):
    
    print(remote.buttons.pressed())
    await sleep(0.5)

# Disconnect
await remote.disconnect()